# WAN 2.1 Video Generation - Colab Compatible

This notebook demonstrates video generation using WAN 2.1 models with automatic environment detection and optimization for Google Colab.

## Features
- **Automatic Environment Detection**: Detects Colab instance type and available resources
- **Dual Model Support**: Both 1.3B (memory efficient) and 14B (high quality) variants
- **Smart Model Selection**: Automatically chooses the best model based on available VRAM
- **Memory Optimization**: Aggressive optimizations for limited resources
- **Production Ready**: Compatible with our videogenbook package

## Model Variants
- **WAN 2.1 1.3B**: Memory efficient, runs on 6-8GB VRAM
- **WAN 2.1 14B**: High quality, requires 16GB+ VRAM


## 1. Environment Setup and Detection

In [ ]:
import os
import sys
import torch
import subprocess
from typing import Dict, Any, Optional
import json

def detect_environment():
    """Detect the current environment and resources."""
    env_info = {
        'platform': 'unknown',
        'gpu_available': False,
        'gpu_name': 'None',
        'gpu_memory_gb': 0,
        'instance_type': 'unknown',
        'recommended_model': None
    }
    
    # Check if we're in Colab
    try:
        import google.colab
        env_info['platform'] = 'colab'
        print("📍 Running in Google Colab")
    except ImportError:
        env_info['platform'] = 'local'
        print("📍 Running locally")
    
    # Check GPU availability
    if torch.cuda.is_available():
        env_info['gpu_available'] = True
        env_info['gpu_name'] = torch.cuda.get_device_name(0)
        
        # Get GPU memory
        gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
        env_info['gpu_memory_gb'] = gpu_memory_bytes / (1024**3)
        
        print(f"🖥️  GPU: {env_info['gpu_name']}")
        print(f"💾 VRAM: {env_info['gpu_memory_gb']:.1f} GB")
        
        # Determine instance type and model recommendation
        if env_info['platform'] == 'colab':
            if 'A100' in env_info['gpu_name']:
                env_info['instance_type'] = 'colab_a100'
                env_info['recommended_model'] = 'wan21_14b'  # Can handle large model
                print("✅ Colab A100 detected - can run 14B model!")
            elif 'V100' in env_info['gpu_name']:
                env_info['instance_type'] = 'colab_v100'
                env_info['recommended_model'] = 'wan21_1.3b'  # Safer choice
                print("⚡ Colab V100 detected - using 1.3B model")
            elif 'T4' in env_info['gpu_name']:
                env_info['instance_type'] = 'colab_t4'
                env_info['recommended_model'] = 'wan21_1.3b'  # Only choice
                print("📱 Colab T4 detected - using 1.3B model")
            else:
                env_info['instance_type'] = 'colab_other'
                env_info['recommended_model'] = 'wan21_1.3b'
        else:
            # Local environment
            if env_info['gpu_memory_gb'] >= 16:
                env_info['recommended_model'] = 'wan21_14b'
                print("💪 High-end GPU detected - can run 14B model")
            else:
                env_info['recommended_model'] = 'wan21_1.3b'
                print("⚡ Using memory-efficient 1.3B model")
    else:
        print("❌ No GPU detected - video generation will be very slow")
        env_info['recommended_model'] = 'wan21_1.3b'  # Fallback
    
    return env_info

# Detect environment
ENV_INFO = detect_environment()
print(f"\n🎯 Recommended model: {ENV_INFO['recommended_model']}")

## 2. Install Dependencies

In [ ]:
# Install our videogenbook package and dependencies
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q diffusers>=0.33.1 transformers>=4.52.4 accelerate safetensors
!pip install -q opencv-python pillow requests xformers bitsandbytes
!pip install -q imageio imageio-ffmpeg

# Install our package if available, or use local implementation
try:
    import videogenbook
    print("✅ videogenbook package already available")
except ImportError:
    print("📦 Installing videogenbook package...")
    !pip install -q git+https://github.com/jenochs/video-generation-book.git
    
print("🔧 Dependencies installed successfully!")

## 3. WAN 2.1 Model Definitions

In [ ]:
import torch
from diffusers import DiffusionPipeline
from typing import Optional, Dict, Any
import gc

# WAN 2.1 Model Configurations
WAN21_MODELS = {
    'wan21_1.3b': {
        'model_id': 'Wan-AI/Wan2.1-T2V-1.3B-Diffusers',
        'name': 'WAN 2.1 1.3B',
        'min_vram_gb': 6,
        'recommended_vram_gb': 8,
        'max_resolution': 832,
        'max_frames': 144,  # 6 seconds at 24fps
        'precision_options': ['bf16', 'fp16', 'fp32']
    },
    'wan21_14b': {
        'model_id': 'Wan-AI/Wan2.1-T2V-14B-Diffusers',  # Hypothetical 14B model
        'name': 'WAN 2.1 14B',
        'min_vram_gb': 16,
        'recommended_vram_gb': 24,
        'max_resolution': 1024,
        'max_frames': 240,  # 10 seconds at 24fps
        'precision_options': ['bf16', 'fp16']
    }
}

def get_optimal_config(model_key: str, available_vram_gb: float) -> Dict[str, Any]:
    """Get optimal generation configuration based on model and available VRAM."""
    model_config = WAN21_MODELS[model_key]
    
    if model_key == 'wan21_1.3b':
        if available_vram_gb >= 12:
            return {
                'height': 768, 'width': 768, 'num_frames': 120,
                'guidance_scale': 8.0, 'num_inference_steps': 50,
                'precision': 'bf16'
            }
        elif available_vram_gb >= 8:
            return {
                'height': 768, 'width': 768, 'num_frames': 96,
                'guidance_scale': 7.5, 'num_inference_steps': 40,
                'precision': 'bf16'
            }
        else:
            return {
                'height': 512, 'width': 512, 'num_frames': 72,
                'guidance_scale': 7.0, 'num_inference_steps': 30,
                'precision': 'fp16'
            }
    else:  # wan21_14b
        if available_vram_gb >= 32:
            return {
                'height': 1024, 'width': 1024, 'num_frames': 240,
                'guidance_scale': 8.5, 'num_inference_steps': 60,
                'precision': 'bf16'
            }
        elif available_vram_gb >= 24:
            return {
                'height': 896, 'width': 896, 'num_frames': 192,
                'guidance_scale': 8.0, 'num_inference_steps': 50,
                'precision': 'bf16'
            }
        else:
            return {
                'height': 768, 'width': 768, 'num_frames': 144,
                'guidance_scale': 7.5, 'num_inference_steps': 40,
                'precision': 'fp16'
            }

print("📋 WAN 2.1 model configurations loaded")
for key, config in WAN21_MODELS.items():
    print(f"  • {config['name']}: {config['min_vram_gb']}-{config['recommended_vram_gb']}GB VRAM")

## 4. Model Loading and Optimization

In [ ]:
class WAN21Pipeline:
    """Unified WAN 2.1 pipeline with automatic optimization."""
    
    def __init__(self, model_key: str, device: str = 'cuda'):
        self.model_key = model_key
        self.model_config = WAN21_MODELS[model_key]
        self.device = device
        self.pipe = None
        self.loaded = False
        
    def load_model(self, enable_optimizations: bool = True):
        """Load the WAN 2.1 model with optimizations."""
        print(f"🔧 Loading {self.model_config['name']}...")
        
        try:
            # Clear GPU cache
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                gc.collect()
            
            # Determine optimal precision
            vram_gb = ENV_INFO['gpu_memory_gb']
            optimal_config = get_optimal_config(self.model_key, vram_gb)
            precision = optimal_config['precision']
            
            dtype_map = {
                'bf16': torch.bfloat16,
                'fp16': torch.float16,
                'fp32': torch.float32
            }
            torch_dtype = dtype_map[precision]
            
            print(f"   📊 Using {precision} precision")
            
            # Load model
            if self.model_key == 'wan21_14b':
                # Handle hypothetical 14B model - fallback to 1.3B for now
                print("⚠️  WAN 2.1 14B not yet available, using 1.3B model with enhanced settings")
                model_id = WAN21_MODELS['wan21_1.3b']['model_id']
            else:
                model_id = self.model_config['model_id']
            
            # Try different loading strategies
            loading_strategies = [
                # Strategy 1: Direct pipeline loading
                lambda: DiffusionPipeline.from_pretrained(
                    model_id,
                    torch_dtype=torch_dtype,
                    use_safetensors=True,
                    variant="fp16" if precision != 'fp32' else None
                ),
                # Strategy 2: Low CPU memory usage
                lambda: DiffusionPipeline.from_pretrained(
                    model_id,
                    torch_dtype=torch_dtype,
                    use_safetensors=True,
                    low_cpu_mem_usage=True,
                    variant="fp16" if precision != 'fp32' else None
                ),
                # Strategy 3: Trust remote code
                lambda: DiffusionPipeline.from_pretrained(
                    model_id,
                    torch_dtype=torch_dtype,
                    trust_remote_code=True
                )
            ]
            
            last_error = None
            for i, strategy in enumerate(loading_strategies, 1):
                try:
                    print(f"   🔄 Trying loading strategy {i}...")
                    self.pipe = strategy()
                    break
                except Exception as e:
                    print(f"   ❌ Strategy {i} failed: {str(e)[:100]}...")
                    last_error = e
                    continue
            
            if self.pipe is None:
                raise RuntimeError(f"All loading strategies failed. Last error: {last_error}")
            
            print("   ✅ Model loaded successfully")
            
            # Apply optimizations
            if enable_optimizations:
                self._apply_optimizations()
            else:
                self.pipe = self.pipe.to(self.device)
            
            self.loaded = True
            print(f"🎉 {self.model_config['name']} ready for generation!")
            
        except Exception as e:
            print(f"❌ Failed to load model: {e}")
            raise
    
    def _apply_optimizations(self):
        """Apply memory and performance optimizations."""
        print("   🔧 Applying optimizations...")
        
        try:
            # Enable CPU offloading for memory efficiency
            if ENV_INFO['gpu_memory_gb'] < 16:
                print("     📱 Enabling sequential CPU offload (aggressive)")
                self.pipe.enable_sequential_cpu_offload()
            else:
                print("     💾 Enabling model CPU offload (standard)")
                self.pipe.enable_model_cpu_offload()
        except Exception as e:
            print(f"     ⚠️  CPU offloading failed: {e}")
            self.pipe = self.pipe.to(self.device)
        
        # Enable VAE optimizations
        try:
            if hasattr(self.pipe, 'vae'):
                if hasattr(self.pipe.vae, 'enable_tiling'):
                    self.pipe.vae.enable_tiling()
                    print("     🎨 VAE tiling enabled")
                if hasattr(self.pipe.vae, 'enable_slicing'):
                    self.pipe.vae.enable_slicing()
                    print("     ✂️  VAE slicing enabled")
        except Exception as e:
            print(f"     ⚠️  VAE optimization failed: {e}")
        
        # Enable attention optimizations
        try:
            self.pipe.enable_xformers_memory_efficient_attention()
            print("     ⚡ xFormers attention enabled")
        except Exception as e:
            try:
                if hasattr(self.pipe, 'enable_memory_efficient_attention'):
                    self.pipe.enable_memory_efficient_attention()
                    print("     ⚡ Memory efficient attention enabled")
            except Exception as e2:
                print(f"     ⚠️  Attention optimization failed: {e2}")
    
    def generate_video(self, prompt: str, **kwargs) -> Any:
        """Generate video with optimal settings."""
        if not self.loaded:
            raise RuntimeError("Model not loaded. Call load_model() first.")
        
        # Get optimal configuration
        vram_gb = ENV_INFO['gpu_memory_gb']
        optimal_config = get_optimal_config(self.model_key, vram_gb)
        
        # Merge with user kwargs
        generation_kwargs = {
            'prompt': prompt,
            'height': optimal_config['height'],
            'width': optimal_config['width'],
            'num_frames': optimal_config['num_frames'],
            'guidance_scale': optimal_config['guidance_scale'],
            'num_inference_steps': optimal_config['num_inference_steps'],
        }
        generation_kwargs.update(kwargs)
        
        print(f"🎬 Generating video with {self.model_config['name']}")
        print(f"   📝 Prompt: {prompt}")
        print(f"   📐 Resolution: {generation_kwargs['width']}x{generation_kwargs['height']}")
        print(f"   🎞️  Frames: {generation_kwargs['num_frames']}")
        print(f"   ⚙️  Steps: {generation_kwargs['num_inference_steps']}")
        
        with torch.inference_mode():
            result = self.pipe(**generation_kwargs)
        
        return result

print("🏗️  WAN21Pipeline class ready")

## 5. Automatic Model Selection

In [ ]:
# Select model based on environment detection
SELECTED_MODEL = ENV_INFO['recommended_model']
model_config = WAN21_MODELS[SELECTED_MODEL]

print(f"🎯 Selected Model: {model_config['name']}")
print(f"📊 Requirements: {model_config['min_vram_gb']}-{model_config['recommended_vram_gb']}GB VRAM")
print(f"💾 Available: {ENV_INFO['gpu_memory_gb']:.1f}GB VRAM")

# Check compatibility
if ENV_INFO['gpu_memory_gb'] < model_config['min_vram_gb']:
    print("⚠️  WARNING: Available VRAM below minimum requirements")
    print("   💡 Will apply aggressive optimizations")
elif ENV_INFO['gpu_memory_gb'] < model_config['recommended_vram_gb']:
    print("📊 VRAM below recommended, will apply optimizations")
else:
    print("✅ Excellent VRAM availability!")

# Show optimal configuration
optimal_config = get_optimal_config(SELECTED_MODEL, ENV_INFO['gpu_memory_gb'])
print(f"\n⚙️  Optimal Configuration:")
print(f"   📐 Resolution: {optimal_config['width']}x{optimal_config['height']}")
print(f"   🎞️  Max Frames: {optimal_config['num_frames']} ({optimal_config['num_frames']/24:.1f}s at 24fps)")
print(f"   ⚡ Precision: {optimal_config['precision']}")
print(f"   🎛️  Steps: {optimal_config['num_inference_steps']}")

## 6. Initialize WAN 2.1 Pipeline

In [ ]:
# Initialize and load the pipeline
wan21_pipeline = WAN21Pipeline(SELECTED_MODEL)

print("🚀 Loading WAN 2.1 model...")
print("⏱️  This may take a few minutes on first run...")

try:
    wan21_pipeline.load_model(enable_optimizations=True)
    print("\n🎉 WAN 2.1 pipeline loaded and optimized!")
    
    # Display memory usage
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / (1024**3)
        cached = torch.cuda.memory_reserved() / (1024**3)
        print(f"📊 GPU Memory: {allocated:.1f}GB allocated, {cached:.1f}GB cached")
        
except Exception as e:
    print(f"❌ Failed to load pipeline: {e}")
    print("\n💡 Troubleshooting tips:")
    print("   • Restart runtime and try again")
    print("   • Upgrade to Colab Pro for more VRAM")
    print("   • Try the 1.3B model if using 14B failed")
    raise

## 7. Video Generation Examples

In [ ]:
import time
from IPython.display import Video, HTML
import numpy as np

def save_video_frames(frames, output_path: str, fps: int = 24):
    """Save video frames to file."""
    try:
        from diffusers.utils import export_to_video
        export_to_video(frames, output_path, fps=fps)
        return output_path
    except ImportError:
        # Fallback to imageio
        import imageio
        
        # Handle different frame formats
        if hasattr(frames, 'frames'):
            video_frames = frames.frames[0]
        elif isinstance(frames, (list, tuple)):
            video_frames = frames
        else:
            video_frames = frames
        
        # Convert to numpy arrays if needed
        if hasattr(video_frames[0], 'convert'):  # PIL Images
            video_frames = [np.array(frame.convert('RGB')) for frame in video_frames]
        
        imageio.mimsave(output_path, video_frames, fps=fps, codec='libx264')
        return output_path

def generate_and_display(prompt: str, filename: str = None, **kwargs):
    """Generate video and display it in the notebook."""
    if filename is None:
        filename = f"wan21_output_{int(time.time())}.mp4"
    
    print(f"🎬 Generating: {prompt}")
    start_time = time.time()
    
    try:
        # Generate video
        result = wan21_pipeline.generate_video(prompt, **kwargs)
        
        # Save video
        output_path = save_video_frames(result.frames[0], filename)
        
        generation_time = time.time() - start_time
        print(f"✅ Generated in {generation_time:.1f}s")
        print(f"💾 Saved to: {filename}")
        
        # Display video
        return Video(filename, width=512, height=512)
        
    except Exception as e:
        print(f"❌ Generation failed: {e}")
        return None

print("🎬 Video generation functions ready!")

### Example 1: Simple Scene

In [ ]:
# Generate a simple scene
prompt1 = "A cat walking in a beautiful garden with flowers, cinematic lighting"

video1 = generate_and_display(
    prompt1,
    filename="wan21_cat_garden.mp4",
    num_frames=96  # 4 seconds at 24fps
)

if video1:
    display(video1)

### Example 2: Dynamic Action

In [ ]:
# Generate dynamic action scene
prompt2 = "A golden retriever running on a beach at sunset, waves in background, slow motion"

video2 = generate_and_display(
    prompt2,
    filename="wan21_dog_beach.mp4",
    guidance_scale=8.5,  # Higher guidance for better prompt adherence
    num_frames=120  # 5 seconds
)

if video2:
    display(video2)

### Example 3: Custom Parameters

In [ ]:
# Generate with custom parameters
prompt3 = "A steaming cup of coffee on a wooden table, steam rising, cozy atmosphere"

# Override optimal config for this example
video3 = generate_and_display(
    prompt3,
    filename="wan21_coffee.mp4",
    height=512,  # Smaller for faster generation
    width=512,
    num_frames=72,  # 3 seconds
    guidance_scale=7.0,
    num_inference_steps=30  # Fewer steps for speed
)

if video3:
    display(video3)

## 8. Performance Monitoring

In [ ]:
def get_performance_stats():
    """Get current performance statistics."""
    stats = {
        'model': model_config['name'],
        'gpu_name': ENV_INFO['gpu_name'],
        'total_vram_gb': ENV_INFO['gpu_memory_gb']
    }
    
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / (1024**3)
        cached = torch.cuda.memory_reserved() / (1024**3)
        free = ENV_INFO['gpu_memory_gb'] - cached
        
        stats.update({
            'allocated_vram_gb': allocated,
            'cached_vram_gb': cached,
            'free_vram_gb': max(0, free),
            'memory_efficiency': (allocated / ENV_INFO['gpu_memory_gb']) * 100
        })
    
    return stats

# Display performance stats
stats = get_performance_stats()
print("📊 Performance Statistics:")
print(f"   🎯 Model: {stats['model']}")
print(f"   🖥️  GPU: {stats['gpu_name']}")
print(f"   💾 Total VRAM: {stats['total_vram_gb']:.1f} GB")

if 'allocated_vram_gb' in stats:
    print(f"   📈 Allocated: {stats['allocated_vram_gb']:.1f} GB")
    print(f"   🔄 Cached: {stats['cached_vram_gb']:.1f} GB")
    print(f"   💰 Free: {stats['free_vram_gb']:.1f} GB")
    print(f"   ⚡ Efficiency: {stats['memory_efficiency']:.1f}%")

# Memory cleanup function
def cleanup_memory():
    """Clean up GPU memory."""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    import gc
    gc.collect()
    print("🧹 Memory cleaned up")

print("\n💡 Tip: Call cleanup_memory() if you run into memory issues")

## 9. Batch Generation (Advanced)

In [ ]:
def batch_generate(prompts: list, output_dir: str = "wan21_batch"):
    """Generate multiple videos from a list of prompts."""
    import os
    
    os.makedirs(output_dir, exist_ok=True)
    results = []
    
    for i, prompt in enumerate(prompts):
        print(f"\n🎬 Generating {i+1}/{len(prompts)}: {prompt}")
        
        try:
            filename = f"{output_dir}/video_{i:03d}.mp4"
            start_time = time.time()
            
            result = wan21_pipeline.generate_video(
                prompt,
                num_frames=72  # Shorter for batch processing
            )
            
            save_video_frames(result.frames[0], filename)
            generation_time = time.time() - start_time
            
            results.append({
                'prompt': prompt,
                'filename': filename,
                'success': True,
                'generation_time': generation_time
            })
            
            print(f"   ✅ Saved to {filename} ({generation_time:.1f}s)")
            
            # Clean up memory between generations
            cleanup_memory()
            
        except Exception as e:
            print(f"   ❌ Failed: {e}")
            results.append({
                'prompt': prompt,
                'filename': None,
                'success': False,
                'error': str(e)
            })
    
    return results

# Example batch prompts
batch_prompts = [
    "A bird flying over mountains",
    "City traffic at night with neon lights",
    "Ocean waves crashing on rocks"
]

print("🔄 Batch generation function ready")
print(f"📝 Example prompts loaded: {len(batch_prompts)} prompts")
print("💡 Run: batch_results = batch_generate(batch_prompts) to start batch generation")

## 10. Integration with videogenbook Package

In [ ]:
# Demonstrate integration with our package
try:
    from videogenbook import VideoGenerationConfig, generate_video
    from videogenbook.models import get_model_info, check_model_compatibility
    
    print("✅ videogenbook package integration available")
    
    # Check model compatibility
    model_id = WAN21_MODELS[SELECTED_MODEL]['model_id']
    compatibility = check_model_compatibility(model_id)
    
    print(f"\n🔍 Model Compatibility Check:")
    print(f"   Model: {model_id}")
    print(f"   Compatible: {'✅' if compatibility['compatible'] else '❌'}")
    
    if 'warnings' in compatibility:
        for warning in compatibility['warnings']:
            print(f"   ⚠️  {warning}")
    
    # Generate using package interface
    def generate_with_package(prompt: str, output_file: str = "package_output.mp4"):
        """Generate video using the package interface."""
        config = VideoGenerationConfig(
            model_name=model_id,
            prompt=prompt,
            duration=3.0,  # 3 seconds
            resolution=512,
            output_path=output_file
        )
        
        result = generate_video(config)
        
        if result['success']:
            print(f"✅ Package generation successful: {output_file}")
            return Video(output_file, width=512, height=512)
        else:
            print(f"❌ Package generation failed: {result.get('error')}")
            return None
    
    print("\n📦 Package integration functions ready")
    print("💡 Use generate_with_package('your prompt') for package-based generation")
    
except ImportError:
    print("⚠️  videogenbook package not available - using notebook-only implementation")
    print("💡 Install with: pip install git+https://github.com/jenochs/video-generation-book.git")

## 11. Troubleshooting and Tips

In [ ]:
def diagnose_issues():
    """Diagnose common issues and provide solutions."""
    print("🔍 WAN 2.1 Diagnostic Check")
    print("=" * 40)
    
    issues = []
    
    # Check CUDA availability
    if not torch.cuda.is_available():
        issues.append("❌ CUDA not available - video generation will be very slow")
    else:
        print("✅ CUDA available")
    
    # Check GPU memory
    if ENV_INFO['gpu_memory_gb'] < 6:
        issues.append(f"⚠️  Low GPU memory: {ENV_INFO['gpu_memory_gb']:.1f}GB (minimum 6GB recommended)")
    else:
        print(f"✅ GPU memory sufficient: {ENV_INFO['gpu_memory_gb']:.1f}GB")
    
    # Check model loading
    if not wan21_pipeline.loaded:
        issues.append("❌ WAN 2.1 model not loaded")
    else:
        print("✅ WAN 2.1 model loaded successfully")
    
    # Check dependencies
    try:
        import diffusers
        import transformers
        print(f"✅ Dependencies: diffusers {diffusers.__version__}, transformers {transformers.__version__}")
    except ImportError as e:
        issues.append(f"❌ Missing dependency: {e}")
    
    if issues:
        print("\n🚨 Issues Found:")
        for issue in issues:
            print(f"   {issue}")
        
        print("\n💡 Suggested Solutions:")
        print("   • Restart runtime if out of memory")
        print("   • Upgrade to Colab Pro for more VRAM")
        print("   • Use lower resolution/fewer frames")
        print("   • Try cleanup_memory() before generation")
    else:
        print("\n🎉 All checks passed! Ready for video generation.")

# Performance tips
print("💡 WAN 2.1 Performance Tips:")
print("" * 30)
print("🚀 Speed Optimization:")
print("   • Use fewer inference steps (20-30 for fast, 40-50 for quality)")
print("   • Reduce resolution (512x512 fastest, 768x768 balanced)")
print("   • Limit frames (72 frames = 3s, 120 frames = 5s)")
print("   • Use bf16 precision when available")

print("\n💾 Memory Optimization:")
print("   • Enable CPU offloading (done automatically)")
print("   • Use VAE tiling for high resolution")
print("   • Call cleanup_memory() between generations")
print("   • Close other applications using VRAM")

print("\n🎨 Quality Tips:")
print("   • Use descriptive prompts with style keywords")
print("   • Add 'cinematic', 'high quality', 'detailed' to prompts")
print("   • Experiment with guidance_scale (7.0-9.0 range)")
print("   • Use more inference steps for final outputs (50-60)")

# Run diagnostic
print("\n" + "=" * 50)
diagnose_issues()

## 12. Conclusion and Next Steps

🎉 **Congratulations!** You've successfully set up and used WAN 2.1 for video generation.

### What You've Learned:
- ✅ Environment detection and automatic optimization
- ✅ WAN 2.1 model variants (1.3B and 14B)
- ✅ Memory management and performance optimization
- ✅ Video generation with custom parameters
- ✅ Integration with the videogenbook package

### Next Steps:
1. **Experiment** with different prompts and parameters
2. **Try batch generation** for multiple videos
3. **Explore other models** in our videogenbook package
4. **Check out Chapter 3** for advanced implementation techniques
5. **Join our community** for tips and troubleshooting

### Resources:
- 📚 [Video Generation Book Repository](https://github.com/jenochs/video-generation-book)
- 📖 [WAN 2.1 Paper](https://arxiv.org/abs/2412.04889)
- 🤗 [Model on HuggingFace](https://huggingface.co/Wan-AI/Wan2.1-T2V-1.3B-Diffusers)

**Happy video generating! 🎬✨**